<a href="https://colab.research.google.com/github/Mystique1337/Developers-Foundary-Fellowship/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Use GPU if available, otherwise fall back to CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Define transformations for the training set.
train_transforms = transforms.Compose([
    transforms.Resize(256),                    # Upscale the image.
    transforms.RandomCrop(224),                # Random crop to 224x224.
    transforms.Lambda(lambda img: img.convert("RGB")),  # Convert grayscale to RGB.
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean.
                         [0.229, 0.224, 0.225])  # ImageNet std.
])

# Define transformations for the validation set.
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Load the MNIST dataset.
# The MNIST dataset will be automatically downloaded if not present.
train_dataset = datasets.MNIST(root='../data', train=True, download=True, transform=train_transforms)
val_dataset = datasets.MNIST(root='../data', train=False, download=True, transform=val_transforms)

# Create data loaders for batching.
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)




Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 52.5MB/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.75MB/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 14.2MB/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.17MB/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
# Load the pre-trained ResNet-18 model.
model = models.resnet18(pretrained=True)

# Freeze all the parameters in the pre-trained model.
for param in model.parameters():
    param.requires_grad = False

# Determine the number of output classes.
num_classes = 10  # MNIST has 10 digits (0-9)

# Get the number of input features for the final fully connected layer.
num_features = model.fc.in_features

# Replace the final fully connected layer with a new one (unfrozen by default).
model.fc = nn.Linear(num_features, num_classes)

# Optionally, unfreeze some deeper layers for fine-tuning.
# For example, to unfreeze the final block:
# for param in model.layer4.parameters():
#     param.requires_grad = True

# Move the model to the configured device.
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 97.8MB/s]


In [ ]:
# Define the loss function.
criterion = nn.CrossEntropyLoss()

# Configure the optimizer to update only the parameters of the final layer.
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# If additional layers were unfrozen, update the optimizer to include them:
# optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)


In [ ]:
num_epochs = 5  # Adjust the number of epochs as needed.

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    print("-" * 20)

    # ----- Training Phase -----
    model.train()  # Set the model to training mode.
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the gradients.
        optimizer.zero_grad()

        # Forward pass.
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimization.
        loss.backward()
        optimizer.step()

        # Accumulate loss and accuracy.
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    print(f"Train Loss: {epoch_loss:.4f}  Train Acc: {epoch_acc:.4f}")

    # ----- Validation Phase -----
    model.eval()  # Set the model to evaluation mode.
    val_loss = 0.0
    val_corrects = 0

    # Disable gradient computation for validation.
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

    epoch_val_loss = val_loss / len(val_dataset)
    epoch_val_acc = val_corrects.double() / len(val_dataset)

    print(f"Val Loss: {epoch_val_loss:.4f}  Val Acc: {epoch_val_acc:.4f}\n")


Epoch 1/5
--------------------
Train Loss: 0.4193  Train Acc: 0.8851
Val Loss: 0.1926  Val Acc: 0.9423

Epoch 2/5
--------------------
Train Loss: 0.2366  Train Acc: 0.9264
